# Testing and outputs

Initial imports

In [1]:
import torch
from model import GPTLanguageModel

In [2]:
#optional device (GPU if available)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cpu


Load the model checkpoint

In [3]:
checkpoint = torch.load("mini_ldr_gpt.pth", map_location=device)

vocab_size = checkpoint["vocab_size"]
block_size = checkpoint["block_size"]
n_embd = checkpoint["n_embd"]
n_head = checkpoint["n_head"]
n_layer = checkpoint["n_layer"]
itos = checkpoint["itos"]
stoi = checkpoint["stoi"]

Rebuild the model and load weights

In [4]:
model = GPTLanguageModel(
    vocab_size=vocab_size,
    block_size=block_size,
    n_embd=n_embd,
    n_head=n_head,
    n_layer=n_layer,
    dropout=0.0,   # no dropout at inference
)
model.load_state_dict(checkpoint["model_state_dict"])
model.to(device)
model.eval()

GPTLanguageModel(
  (token_embedding_table): Embedding(95, 128)
  (position_embedding_table): Embedding(128, 128)
  (blocks): Sequential(
    (0): Block(
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0-3): 4 x Head(
            (key): Linear(in_features=128, out_features=32, bias=False)
            (query): Linear(in_features=128, out_features=32, bias=False)
            (value): Linear(in_features=128, out_features=32, bias=False)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (proj): Linear(in_features=128, out_features=128, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (ffwd): FeedForward(
        (net): Sequential(
          (0): Linear(in_features=128, out_features=512, bias=True)
          (1): ReLU()
          (2): Linear(in_features=512, out_features=128, bias=True)
          (3): Dropout(p=0.0, inplace=False)
        )
      )
      (ln1): LayerNorm((128,), eps=1e-05, elementwise_affine=T

Encoding and decoding

In [5]:
encode = lambda s: [stoi[c] for c in s if c in stoi]
decode = lambda l: "".join([itos[i] for i in l])

Some example outputs

In [9]:

device = "cpu"

# example prompts to test the trained Taylor Swift model
#more examples for both models in the repository, under ldr_outputs.txt and taylor_outputs.txt
example_prompts = ["red", "lover", "guitar", "fearless", "reputation", "America", "New York", "night"]

for prompt in example_prompts:
    print(f"\nPrompt: {prompt}")

    context = torch.tensor([encode(prompt)], dtype=torch.long, device=device)
    output = model.generate(context, max_new_tokens=300)

    generated_text = decode(output[0].tolist())
    print(generated_text)



Prompt: red
red, ' (Darling, la-la-la, lroom)', 'With River, standines I was the floor reason', 'Your way Kowling out of decision', "It's you're here thing, get all high", 'Jimmy only?', 'Ah', 'Hmm-hmm-hmm-hmm', "No, babe, I love you don't wanna wanna was really", "And I'm lonely", 'And your knees', 'And you love 

Prompt: lover
lover singer', "There's singing off the new ach here and rown", "Gramma said that she'll let a come", "You're hod for me", 'There’s in the greas', 'Look for the worfume cher and on the ceath new', 'And women, feel it all of love out of my day', 'Back to someone the choose of and his us like a L.L)', 'I g

Prompt: guitar
guitars those I homan a lost op', 'I late everything in\', "Dream dreaming everywhere and I can't get high", 'I did it, heart me my memost of you', "But I'm pretter, just throught", "Thrad the touch untry, the one if I tell my had reason am wavin' Reason", "Oh, God just just a leave man", 'I know do it it

Prompt: fearless
fearless blue', "And 